In [1]:
dataset_path = '../Prostate Dataset'
out_dir_img = '../output_dir/images'
out_dir_mask = '../output_dir/masks'
temp_dir = '../temp/'

In [6]:
from platipy.imaging import ImageVisualiser
from platipy.imaging.tests.data import download_and_extract_zip_file
from platipy.dicom.io.rtstruct_to_nifti import convert_rtstruct
from rt_utils import RTStructBuilder
import os

from rt_utils import RTStructBuilder
import SimpleITK as sitk

def read_dicom_image_to_sitk(img_path):
  reader = sitk.ImageSeriesReader()

  dicom_names = reader.GetGDCMSeriesFileNames(str(img_path))
  reader.SetFileNames(dicom_names)

  image = reader.Execute()

  return image

def extracted_prostate(rtstruct_path, dicom_path, ROI_name, out_path, index):
    # Load existing RT Struct. Requires the series path and existing RT Struct path
    rtstruct = RTStructBuilder.create_from(
      dicom_series_path=dicom_path, 
      rt_struct_path=rtstruct_path
    )

    prostate_rtstruct = RTStructBuilder.create_new(
      dicom_series_path=dicom_path,
    )


    # Loading the 3D Mask from within the RT Struct
    mask_3d = rtstruct.get_roi_mask_by_name(ROI_name)

    prostate_rtstruct.add_roi(mask_3d, name=ROI_name)
    
    location = f'mask_{index}'
    output = os.path.join(out_path, location)
    prostate_rtstruct.save(output)
    
    return output + '.dcm'
  

folders = [name for name in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, name))]


for index,folder in enumerate(folders):
    first_level_folder_path = os.path.join(dataset_path, folder)
    
    # Find second-level folders within each first-level folder
    second_level_folders = [name for name in os.listdir(first_level_folder_path) if os.path.isdir(os.path.join(first_level_folder_path, name))]

    # Loop over each second-level folder
    for subfolder in second_level_folders:
        second_level_folder_path = os.path.join(first_level_folder_path, subfolder)
        
        # Initialize paths for contour and image data
        contour_path = None
        image_path = None
        
        # Search for contour and image files in the second-level folder
        for file_name in os.listdir(second_level_folder_path):
            file_path = os.path.join(second_level_folder_path, file_name)
            
            # Adjust these conditions based on file naming
            if os.path.isdir(file_path) and 'contouring' in file_name.lower():
                for contour_file in os.listdir(file_path):
                    contour_path = os.path.join(file_path, contour_file)
                    break

            elif 'pelvis' in file_name.lower():
                image_path = file_path
        
        # Print or store the paths
        if contour_path and image_path:

            rtstruct_file = extracted_prostate(contour_path,image_path,'Prostate', temp_dir, index)
            mask = f'mask{index}'
            convert_rtstruct(dcm_img = image_path, dcm_rt_file = rtstruct_file, prefix=mask, output_dir= out_dir_mask)
            
            image_data = read_dicom_image_to_sitk(image_path)
            out_file = os.path.join(out_dir_img, f'image_{index}.nii.gz')
            sitk.WriteImage(image_data, str(out_file))
            

Writing file to ../temp/mask_0.dcm
